<a href="https://colab.research.google.com/github/taka-onohara/google-colab/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'emp', 'labsh','avh']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Y = rgdpna, K = rkna, L = emp (* avh?), alpha = 1 - labsh then fix by country, A = (the rest)
data['Y'] = data['rgdpna']
data['K'] = data['rkna']
data['L'] = data['emp']
data['alpha'] = 1 - data['labsh']
data['alpha'] = data.groupby('countrycode')['alpha'].transform('mean')
data['A'] = data['Y'] / ((data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha'])))

data['test'] = data['A'] * (data['K'] ** data['alpha']) * (data['L'] ** (1 - data['alpha']))
data.drop(data.columns[[2,3,4,5,6]], axis=1, inplace=True)
# print(data.head(10))

# y = Y/L, k = K/L, ln(y) = ln(A) + alpha * ln(k)
data['ln(y)'] = np.log(data['Y'] / data['L'])
data['ln(A)'] = np.log(data['A'])
data['ln(k)'] = np.log(data['K'] / data['L'])

data['test1'] = data['ln(A)'] + data['alpha'] * data['ln(k)']
#print(data.head(10))

data = data.sort_values('year')
grouped_data = data.groupby('countrycode')

data['growth_rate'] = grouped_data['ln(y)'].diff() * 100
data['TFP_growth'] = grouped_data['ln(A)'].diff() * 100
data['capital_deepening'] = data['alpha'] * (grouped_data['ln(k)'].diff() * 100)

data = data.dropna()

data['test2'] = data['TFP_growth'] + data['capital_deepening']
print(data.head(10))

data.rename(columns = {'alpha':'capital_share'}, inplace = True)
data['TFP_share'] = 1 - data['capital_share']

summary = data.groupby('countrycode').agg({'growth_rate': 'mean',
                                           'TFP_growth':'mean',
                                           'capital_deepening':'mean',
                                           'TFP_share':'mean',
                                           'capital_share':'mean'})
#print('---summary---')
#print(summary)

overall = summary.mean()
#print('---overall average---')
#print(overall)

       year countrycode             Y         K           L     alpha  \
5926   1996         ITA  2.170005e+06  0.689952   22.114836  0.484425   
4316   1996         GBR  1.885852e+06  0.604429   25.924555  0.414704   
12156  1996         USA  1.188185e+07  0.539946  129.357666  0.393235   
4176   1996         FRA  2.043539e+06  0.619433   23.806284  0.381140   
6136   1996         JPN  4.285729e+06  0.719400   67.725853  0.429427   
2146   1996         CAN  1.076842e+06  0.486632   13.636177  0.345020   
3266   1996         DEU  3.132657e+06  0.673956   37.807400  0.369455   
4177   1997         FRA  2.091282e+06  0.633773   24.003853  0.381140   
6137   1997         JPN  4.331846e+06  0.750685   68.184273  0.429427   
3267   1997         DEU  3.188799e+06  0.692014   37.770332  0.369455   

                  A          test      ln(y)      ln(A)     ln(k)      test1  \
5926   526326.87500  2.170006e+06  11.493992  13.173677 -3.467383  11.493991   
4316   345736.81250  1.885852e+06  1

C:\Users\tonoh\AppData\Local\Temp\ipykernel_11032\297064305.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['test2'] = data['TFP_growth'] + data['capital_deepening']
C:\Users\tonoh\AppData\Local\Temp\ipykernel_11032\297064305.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns = {'alpha':'capital_share'}, inplace = True)
C:\Users\tonoh\AppData\Local\Temp\ipykernel_11032\297064305.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead
